In [3]:

# coding: utf-8

# In[225]:


import numpy as np
import pandas as pandas
import requests
import sqlite3
from datetime import date

from bs4 import BeautifulSoup

def getArray(url): 
    page = requests.get(url)
    y = []
    soup1 = BeautifulSoup(page.content, 'html.parser')
    for tr in soup1.find_all('tr'):
        for td in tr.find_all('td'):
    
            y.append(td.get_text())
        
    games = np.array(y).reshape(int(len(y)/21),21)
    #print(games)
    return(games)
    

def makeDF(data):
    currentDate = date.today()
    columns = ['Rank','team','conf','w-l','adjEM','adjO','adjO_seed','adjD','adjD_seed','adjT','adjT_seed','luck','luck_seed','sched','sched_seed','OppO', 'OppO_seed','OppD','OppD_seed','ncadjEM','ncadjEM_seed']
    df = pandas.DataFrame(data, columns = columns)
    datevec = pandas.DataFrame([{'date': currentDate}] * len(df)) 
    df = df.join(datevec, how = 'left')
    return df


conn = sqlite3.connect('kenpom.db')
c = conn.cursor()
sqlcode = """
CREATE TABLE If NOT EXISTS STATS( 
    Rank int,
    team varchar(22),
    conf text,
    `w-l` text,
    adjEM real,
    adjO real,
    adjO_seed int,
    adjD real,
    adjD_seed int,
    adjT real,
    adjT_seed int,
    luck real,
    luck_seed int,
    sched real,
    sched_seed int,
    OppO real,
    OppO_seed int,
    OppD real,
    OppD_seed int,
    ncadjEM real,
    ncadjem_seed int,
    date date,
    PRIMARY KEY(team, date)
)
"""
conn.commit()
#c.execute(sqlcode)    
    


In [4]:
games2021 = getArray("https://kenpom.com/index.php")
mydf = makeDF(games2021)
print(mydf)
mydf.to_sql('STATS', conn, if_exists = 'append', index = False)


    Rank                     team  conf    w-l   adjEM   adjO adjO_seed  \
0      1                Gonzaga 1   WCC   26-0  +36.59  126.1         1   
1      2               Michigan 1   B10   20-4  +30.70  119.5         6   
2      3               Illinois 1   B10   23-6  +30.57  119.0         8   
3      4                 Baylor 1   B12   22-2  +29.57  123.4         3   
4      5                Houston 2  Amer   24-3  +28.89  119.0         7   
5      6                   Iowa 2   B10   21-8  +28.86  123.6         2   
6      7               Ohio St. 2   B10   21-9  +25.38  122.3         4   
7      8                Alabama 2   SEC   24-6  +25.07  111.8        34   
8      9         Loyola Chicago 8   MVC   24-4  +23.77  110.5        49   
9     10               Virginia 4   ACC   18-6  +22.63  115.7        12   
10    11              Wisconsin 9   B10  17-12  +22.62  112.6        32   
11    12              Villanova 5    BE   16-6  +22.60  118.7         9   
12    13                 

In [7]:
mydf.to_csv('kenpom.csv', mode = 'w', header = True, index = False)


KeyError: (0, 1)